## Build Functions for ELT

## Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 uninstall pandas pyarrow fireducks numpy -y
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppresses INFO and WARNING messages
import tensorflow as tf

from src import modules as f # f is for function
%load_ext autoreload
%autoreload 2

# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

E0000 00:00:1733826811.208526   27407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733826811.211676   27407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Num GPUs Available: 1


### Download, Transform, and Modeling All in One

In [12]:
f.predictions('avgo')

AVGO 5m Interval Timestamp: 2024-12-15 02:28:40 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,static
kelly_1:2.5,0.116269,0.101717,0.151064,0.127939
prob_up,0.194732,0.271755,0.22176,0.125
prob_static,0.229616,0.358367,0.338596,0.5
prob_down,0.575652,0.369878,0.439645,0.375
precision,0.368764,0.358369,0.393617,0.377099
recall,0.354167,0.347917,0.385417,0.515658
f1,0.361318,0.353066,0.389474,0.435626
support,"[479.0, 479.0, 480.0]","[479.0, 479.0, 480.0]","[479.0, 479.0, 480.0]","[479.0, 479.0, 480.0]"


AVGO 15m Interval Timestamp: 2024-12-15 02:28:43 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,up,down
kelly_1:2.5,0.186486,0.282412,0.16,0.145133
prob_up,0.391388,0.310926,0.359008,0.125
prob_static,0.279861,0.35079,0.314235,0.25
prob_down,0.328751,0.338284,0.326757,0.625
precision,0.418919,0.487437,0.4,0.389381
recall,0.380368,0.587879,0.368098,0.266667
f1,0.398714,0.532967,0.383387,0.316547
support,"[163.0, 165.0, 165.0]","[163.0, 165.0, 165.0]","[163.0, 165.0, 165.0]","[163.0, 165.0, 165.0]"


AVGO 1h Interval Timestamp: 2024-12-15 02:28:44 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,down,up
kelly_1:2.5,0.140249,0.062555,0.075472,0.051793
prob_up,0.393893,0.298037,0.337546,0.5
prob_static,0.300478,0.341185,0.32321,0.25
prob_down,0.305629,0.360778,0.339243,0.25
precision,0.385892,0.330396,0.339623,0.322709
recall,0.366142,0.296443,0.284585,0.318898
f1,0.375758,0.3125,0.309677,0.320792
support,"[254.0, 256.0, 253.0]","[254.0, 256.0, 253.0]","[254.0, 256.0, 253.0]","[254.0, 256.0, 253.0]"


AVGO 1wk Interval Timestamp: 2024-12-15 02:28:50 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,up,down,up
kelly_1:2.5,0.095833,0.134545,0.0,0.12093
prob_up,0.379998,0.42389,0.307358,0.375
prob_static,0.202295,0.25671,0.299875,0.25
prob_down,0.417707,0.3194,0.392767,0.375
precision,0.354167,0.381818,0.285714,0.372093
recall,0.326923,0.411765,0.192308,0.313725
f1,0.34,0.396226,0.229885,0.340426
support,"[51.0, 51.0, 52.0]","[51.0, 51.0, 52.0]","[51.0, 51.0, 52.0]","[51.0, 51.0, 52.0]"


AVGO 1mo Interval Timestamp: 2024-12-15 02:28:50 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,down,down
kelly_1:2.5,-0.2,0.363636,-0.018182,0.16
prob_up,0.06307,0.202297,0.283277,0.125
prob_static,0.203035,0.443475,0.311126,0.25
prob_down,0.733895,0.354228,0.405596,0.625
precision,0.142857,0.545455,0.272727,0.4
recall,0.1,0.6,0.3,0.2
f1,0.117647,0.571429,0.285714,0.266667
support,"[11.0, 10.0, 10.0]","[11.0, 10.0, 10.0]","[11.0, 10.0, 10.0]","[11.0, 10.0, 10.0]"


In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')